### Corrupting collected data 

In [499]:
import pandas as pd
import numpy as np
import csv
import shutil
import random
import re

In [507]:
'''
# read the csv file
df = pd.read_csv(r'..\openfield-Pranav-2018-08-20\labeled-data-correct\m1s1\CollectedData_Pranav.csv')
print(df)

df = df[2:] # we don't want later to select one of the 3 first rows as they're not data

# randomly select 10 rows of data
data = df.sample(n = 10).sort_index()
'''

"\n# read the csv file\ndf = pd.read_csv(r'..\\openfield-Pranav-2018-08-20\\labeled-data-correct\\m1s1\\CollectedData_Pranav.csv')\nprint(df)\n\ndf = df[2:] # we don't want later to select one of the 3 first rows as they're not data\n\n# randomly select 10 rows of data\ndata = df.sample(n = 10).sort_index()\n"

In [500]:
'''
import deeplabcut

cfg = deeplabcut.auxiliaryfunctions.read_config(r'..\openfield-Pranav-2018-08-20\config.yaml')
scorer = cfg['scorer']
odyparts = cfg['bodyparts']
#columnindex = pd.MultiIndex.from_product([[scorer], bodyparts, ['x', 'y']], names=['scorer', 'bodyparts', 'coords'])

# assign random values to later replace the data
corruptedData = pd.DataFrame(np.random.rand(10, 8) * 500, columns=columnindex).astype(object)
corruptedData.index = data.index
corruptedData.insert(0, "0", data.iloc[:,0])
corruptedData.columns = ["0", "1","2", "3","4","5","6","7","8"]


# we want to know which images have been corrupted to later on analyse them
corruptedImages = corruptedData.index - 3

print(corruptedData)
print(corruptedImages)
#corruptedData.dtypes
'''

In [508]:
original = r'..\openfield-Pranav-2018-08-20\labeled-data-correct\m1s1\CollectedData_Pranav.csv'
target = r'..\openfield-Pranav-2018-08-20\labeled-data\m1s1\CollectedData_Pranav.csv'
shutil.copyfile(original, target)

df = pd.read_csv(r'..\openfield-Pranav-2018-08-20\labeled-data\m1s1\CollectedData_Pranav.csv')

columns = df.columns.tolist()
columns.pop(0)

filename = df.loc[:,"scorer"].tolist()
filename.pop(0);filename.pop(0)

random.shuffle(filename)

for name in filename[:10]:
    string = "".join(re.findall(r'\d',name))
    id = int(string[2:])
    for column in columns:
        df.iloc[id][column] = np.random.uniform(5,500)

df.to_csv(r'..\openfield-Pranav-2018-08-20\labeled-data\m1s1\CollectedData_Pranav.csv', index = False)
df.astype(str).to_hdf(r'..\openfield-Pranav-2018-08-20\labeled-data\m1s1\CollectedData_Pranav.h5',  "df_with_missing", format="table",mode="w")

In [502]:
# replace in a new file the correct values with the corrupted ones
"""
original = r'..\openfield-Pranav-2018-08-20\labeled-data-correct\m1s1\CollectedData_Pranav.csv'
target = r'..\openfield-Pranav-2018-08-20\labeled-data\m1s1\CollectedData_Pranav.csv'
shutil.copyfile(original, target)

df = pd.read_csv(r'..\openfield-Pranav-2018-08-20\labeled-data\m1s1\CollectedData_Pranav.csv', header = None)

print(df.columns)
df.iloc[corruptedData.index,:] = corruptedData[:]
print(df.head(10
             ))
# swap left and right ear for some of the data (maybe find better way to do this, def f?)
swappedEars = random.sample(range(5, 100), 5)
swappedEars[:] = [number - 3 for number in swappedEars] # we want the right image number
print(swappedEars)

def swapData(df, sample, i, j):
    tmp = df.iloc[sample, i]
    df.loc[sample, i] = df.loc[sample, j]
    df.loc[sample, j] = tmp
    return df

df = swapData(df,swappedEars,3,5)
df = swapData(df,swappedEars,4,6)
print(df.columns)
df.to_csv(r'..\openfield-Pranav-2018-08-20\labeled-data\m1s1\CollectedData_Pranav.csv', index = False)
df.astype(str).to_hdf(r'..\openfield-Pranav-2018-08-20\labeled-data\m1s1\CollectedData_Pranav.h5',  "df_with_missing", format="table",mode="w")
"""

'\noriginal = r\'..\\openfield-Pranav-2018-08-20\\labeled-data-correct\\m1s1\\CollectedData_Pranav.csv\'\ntarget = r\'..\\openfield-Pranav-2018-08-20\\labeled-data\\m1s1\\CollectedData_Pranav.csv\'\nshutil.copyfile(original, target)\n\ndf = pd.read_csv(r\'..\\openfield-Pranav-2018-08-20\\labeled-data\\m1s1\\CollectedData_Pranav.csv\', header = None)\n\nprint(df.columns)\ndf.iloc[corruptedData.index,:] = corruptedData[:]\nprint(df.head(10\n             ))\n# swap left and right ear for some of the data (maybe find better way to do this, def f?)\nswappedEars = random.sample(range(5, 100), 5)\nswappedEars[:] = [number - 3 for number in swappedEars] # we want the right image number\nprint(swappedEars)\n\ndef swapData(df, sample, i, j):\n    tmp = df.iloc[sample, i]\n    df.loc[sample, i] = df.loc[sample, j]\n    df.loc[sample, j] = tmp\n    return df\n\ndf = swapData(df,swappedEars,3,5)\ndf = swapData(df,swappedEars,4,6)\nprint(df.columns)\ndf.to_csv(r\'..\\openfield-Pranav-2018-08-20\\lab

In [503]:
#bodyparts = df.columns
#bodyparts

In [504]:
import deeplabcut

#check correct labels using deeplabcut check_labels
deeplabcut.check_labels(r'..\openfield-Pranav-2018-08-20\config.yaml')

Creating images with labels by Pranav.


KeyError: 'Requested level (bodyparts) does not match index name (None)'

In [505]:
""""
# look for frames that have abnormal distances between bodyparts (here snout and tail)
treshold = 100
df = pd.read_hdf('..\openfield-Pranav-2018-08-20\labeled-data\m1s1\CollectedData_Pranav.h5',key='')
snout = df.xs('snout', level='bodyparts', axis=1).to_numpy()
tail = df.xs('tail', level='bodyparts', axis=1).to_numpy()
diff = (snout-tail).reshape(len(df), -1, 2)
dist = np.linalg.norm(diff, axis=2)
mask = np.any(dist >= treshold, axis=1)
abnormalFrames = df.iloc[mask].index
print(abnormalFrames)
"""

'"\n# look for frames that have abnormal distances between bodyparts (here snout and tail)\ntreshold = 100\ndf = pd.read_hdf(\'..\\openfield-Pranav-2018-08-20\\labeled-data\\m1s1\\CollectedData_Pranav.h5\',key=\'\')\nsnout = df.xs(\'snout\', level=\'bodyparts\', axis=1).to_numpy()\ntail = df.xs(\'tail\', level=\'bodyparts\', axis=1).to_numpy()\ndiff = (snout-tail).reshape(len(df), -1, 2)\ndist = np.linalg.norm(diff, axis=2)\nmask = np.any(dist >= treshold, axis=1)\nabnormalFrames = df.iloc[mask].index\nprint(abnormalFrames)\n'

In [506]:
# look for frames where bodyparts (like right and left ear) might have been swapped
